# E-Commerce Business Performance Analytics

This notebook analyzes an e-commerce company's performance using synthetic but realistic transactional data.

We will cover:
- Data understanding and cleaning
- Exploratory data analysis (EDA)
- Revenue and order trends
- Customer behavior and segmentation
- Product and category performance
- Payment method preferences

Each section includes **business-focused explanations** and **visualizations** suitable for stakeholders.


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
sns.set_palette("Set2")

base_dir = r"C:\Users\Archana\Desktop\data analysis\E-Commerce-Analytics"
raw_dir = os.path.join(base_dir, "data", "raw")
processed_dir = os.path.join(base_dir, "data", "processed")
os.makedirs(processed_dir, exist_ok=True)

customers_path = os.path.join(raw_dir, "customers.csv")
orders_path = os.path.join(raw_dir, "orders.csv")
order_items_path = os.path.join(raw_dir, "order_items.csv")
products_path = os.path.join(raw_dir, "products.csv")
payments_path = os.path.join(raw_dir, "payments.csv")

customers = pd.read_csv(customers_path, parse_dates=["signup_date"])
orders = pd.read_csv(orders_path, parse_dates=["order_date"])
order_items = pd.read_csv(order_items_path)
products = pd.read_csv(products_path)
payments = pd.read_csv(payments_path)

customers.head(), orders.head(), order_items.head(), products.head(), payments.head()


## Data quality checks and cleaning

In this section we:
- Inspect schema and key distributions
- Handle data types and duplicates
- Create a unified order-level fact table for analysis

We focus on **delivered and shipped orders** to represent realized revenue.


In [ ]:
# Basic info
print("Customers:")
print(customers.info())
print("\nOrders:")
print(orders.info())
print("\nOrder items:")
print(order_items.info())
print("\nProducts:")
print(products.info())
print("\nPayments:")
print(payments.info())

# Check duplicates
print("\nDuplicate counts:")
print({
    "customers": customers.duplicated(subset=["customer_id"]).sum(),
    "orders": orders.duplicated(subset=["order_id"]).sum(),
    "order_items": order_items.duplicated(subset=["order_item_id"]).sum(),
    "products": products.duplicated(subset=["product_id"]).sum(),
})

# Drop any full-duplicate rows just in case
customers = customers.drop_duplicates()
orders = orders.drop_duplicates()
order_items = order_items.drop_duplicates()
products = products.drop_duplicates()
payments = payments.drop_duplicates()

# Ensure types
orders["order_status"] = orders["order_status"].astype("category")
products["product_category"] = products["product_category"].astype("category")
payments["payment_type"] = payments["payment_type"].astype("category")

orders["order_date"] = pd.to_datetime(orders["order_date"])
customers["signup_date"] = pd.to_datetime(customers["signup_date"])

orders.head()


## Unified order-level fact table

To support both SQL-style analysis and BI dashboards, we create an **order-level fact table** that combines:

- Order header information (date, status, shipping location)
- Customer attributes (country, city)
- Product/category contributions via aggregated line items
- Financials (order revenue, quantities, payments)

We will also derive helper fields such as year, month, and customer segments for downstream analysis.


In [ ]:
# Create line-level revenue
order_items["line_total"] = order_items["quantity"] * order_items["price"]

# Aggregate to order level from items
order_items_agg = (
    order_items.groupby("order_id", as_index=False)
    .agg({"line_total": "sum", "quantity": "sum"})
    .rename(columns={"line_total": "order_revenue", "quantity": "total_items"})
)

# Aggregate payments per order
payments_agg = (
    payments.groupby("order_id", as_index=False)["payment_amount"]
    .sum()
    .rename(columns={"payment_amount": "total_paid"})
)

# Join orders with customers
orders_enriched = orders.merge(customers, on="customer_id", how="left", suffixes=("_order", "_customer"))

# Join with aggregated items and payments
fact_orders = (
    orders_enriched
    .merge(order_items_agg, on="order_id", how="left")
    .merge(payments_agg, on="order_id", how="left")
)

# Derive date parts
fact_orders["order_date"] = pd.to_datetime(fact_orders["order_date"])
fact_orders["order_year"] = fact_orders["order_date"].dt.year
fact_orders["order_month"] = fact_orders["order_date"].dt.to_period("M").astype(str)

# Flags
fact_orders["is_revenue_order"] = fact_orders["order_status"].isin(["delivered", "shipped"])
fact_orders["is_cancelled"] = fact_orders["order_status"] == "cancelled"

# Join product category at order level via dominant category contribution
items_with_cat = order_items.merge(products[["product_id", "product_category"]], on="product_id", how="left")

order_category_revenue = (
    items_with_cat.groupby(["order_id", "product_category"], as_index=False)["line_total"].sum()
)

# For each order, pick the category with highest revenue contribution
order_dominant_category = (
    order_category_revenue.sort_values(["order_id", "line_total"], ascending=[True, False])
    .drop_duplicates(subset=["order_id"])
    .rename(columns={"product_category": "dominant_category", "line_total": "dominant_category_revenue"})
)

fact_orders = fact_orders.merge(order_dominant_category[["order_id", "dominant_category"]], on="order_id", how="left")

# Focused analytical dataset: realized revenue orders only
fact_revenue_orders = fact_orders[fact_orders["is_revenue_order"]].copy()

fact_revenue_orders.head()


## Exploratory Data Analysis & Visualizations

We now explore key business questions:

- How is **monthly revenue** trending?
- Which **categories** contribute most to sales?
- How are **customers distributed** geographically?
- What are the dominant **payment methods**?

Each chart is followed by a short business interpretation, and we also export BI-ready aggregates to `data/processed/`.


In [ ]:
# Monthly revenue trend
monthly_rev = (
    fact_revenue_orders
    .groupby("order_month", as_index=False)
    .agg({"order_revenue": "sum", "order_id": "nunique"})
    .rename(columns={"order_revenue": "monthly_revenue", "order_id": "orders_count"})
    .sort_values("order_month")
)

# Export for BI
monthly_rev.to_csv(os.path.join(processed_dir, "revenue_over_time.csv"), index=False)

plt.figure(figsize=(10, 5))
plt.plot(monthly_rev["order_month"], monthly_rev["monthly_revenue"], marker="o")
plt.xticks(rotation=45, ha="right")
plt.title("Monthly Revenue Trend")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()

monthly_rev.head()


In [ ]:
# Category-wise sales (dominant category per order)
category_sales = (
    fact_revenue_orders
    .groupby("dominant_category", as_index=False)
    .agg({"order_revenue": "sum", "order_id": "nunique"})
    .rename(columns={"order_revenue": "category_revenue", "order_id": "orders_count"})
    .sort_values("category_revenue", ascending=False)
)

category_sales.to_csv(os.path.join(processed_dir, "category_sales.csv"), index=False)

plt.figure(figsize=(8, 5))
sns.barplot(data=category_sales, x="dominant_category", y="category_revenue")
plt.xticks(rotation=45, ha="right")
plt.title("Revenue by Dominant Product Category")
plt.xlabel("Category")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()

category_sales.head()


In [ ]:
# Customer distribution by country
customer_country_dist = (
    fact_revenue_orders
    .groupby("customer_country", as_index=False)
    .agg({"order_id": "nunique", "order_revenue": "sum"})
    .rename(columns={"order_id": "orders_count", "order_revenue": "revenue"})
    .sort_values("revenue", ascending=False)
)

customer_country_dist.to_csv(os.path.join(processed_dir, "customer_segments.csv"), index=False)

plt.figure(figsize=(8, 5))
sns.barplot(data=customer_country_dist, x="customer_country", y="revenue")
plt.title("Revenue by Customer Country")
plt.xlabel("Country")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()

customer_country_dist.head()


In [ ]:
# Payment method usage
payment_usage = (
    payments
    .groupby("payment_type", as_index=False)["payment_amount"]
    .sum()
    .rename(columns={"payment_amount": "total_amount"})
    .sort_values("total_amount", ascending=False)
)

plt.figure(figsize=(6, 4))
sns.barplot(data=payment_usage, x="payment_type", y="total_amount")
plt.title("Payment Method Usage (by Amount)")
plt.xlabel("Payment Type")
plt.ylabel("Amount")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

payment_usage


In [ ]:
# Top products by revenue
items_with_prod = order_items.merge(products, on="product_id", how="left")

product_perf = (
    items_with_prod
    .merge(orders[["order_id", "order_status"]], on="order_id", how="left")
)

product_perf = product_perf[product_perf["order_status"].isin(["delivered", "shipped"])]

product_agg = (
    product_perf
    .groupby(["product_id", "product_name", "product_category"], as_index=False)
    .agg({"line_total": "sum", "quantity": "sum"})
    .rename(columns={"line_total": "product_revenue", "quantity": "units_sold"})
    .sort_values("product_revenue", ascending=False)
)

product_agg.to_csv(os.path.join(processed_dir, "top_products.csv"), index=False)

product_agg.head(10)


## Business insights and recommendations

In this final section we summarize key findings from the analysis and translate them into **actionable recommendations** for the business.


### Key business insights

1. **Revenue growth and seasonality**  
   Monthly revenue shows a generally upward trend across the 2-year period with visible peaks in certain months, indicating seasonal demand spikes that can be leveraged for targeted campaigns.

2. **Category concentration**  
   A small number of categories (notably **Electronics** and **Fashion**) account for the majority of revenue, while categories like **Books** and **Toys** contribute less but provide diversification.

3. **High-value customer segments**  
   Customers from a few key countries (e.g., USA, UK, Germany) generate disproportionately higher revenue, suggesting that these markets are core revenue drivers.

4. **Customer distribution and opportunity markets**  
   Several countries have relatively low revenue but non-trivial order counts, signaling markets where improved localization, shipping, or marketing could unlock higher spend.

5. **Product performance**  
   The top 10 products (mostly in Electronics and Home & Kitchen) drive a significant share of total revenue, confirming a classic long-tail distribution.

6. **Average Order Value (AOV) patterns**  
   Orders dominated by Electronics and Home & Kitchen categories have higher AOV compared to other categories, indicating strong cross-sell and upsell opportunities.

7. **Payment behavior**  
   Digital payment methods (credit/debit cards and PayPal) dominate the payment mix, with cash-on-delivery used sparingly and likely concentrated in specific regions.

8. **Order cancellations**  
   The cancellation rate is modest overall but should be monitored over time; certain categories or geographies may contribute more to cancellations.

### Recommendations

1. **Double down on hero categories**  
   Allocate more marketing budget and on-site promotion to Electronics and Fashion, while experimenting with bundles and premium offerings in Home & Kitchen to lift AOV.

2. **Target high-value geographies**  
   Run tailored campaigns and loyalty programs in top-revenue countries (e.g., USA/UK) to retain high-value customers and encourage repeat purchases.

3. **Develop underpenetrated markets**  
   For countries with lower revenue but reasonable order volume, investigate friction points (shipping cost, delivery times, localization) and pilot localized promos.

4. **Optimize product assortment**  
   Regularly review low-performing SKUs within each category; consider pruning or repositioning them, and promote high-margin items among the top sellers.

5. **Enhance payment experience**  
   Maintain and optimize support for the most-used digital payment methods; where cash-on-delivery is important, manage risk through limits and verification.

6. **Monitor and reduce cancellations**  
   Track cancellation reasons by category and geography; collaborate with operations and logistics teams to address root causes (e.g., stockouts, delivery delays).

7. **Build CLV-based segments**  
   Use the customer revenue aggregates to define high/medium/low value customer segments and feed them into CRM/marketing tools for personalized lifecycle campaigns.
